In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset


In [3]:
df = pd.read_csv("C:\\Users\\cheta\\Downloads\\poems-100.csv")

text = " ".join(df["text"].astype(str)).lower()
tokens = text.split()







In [4]:
vocab = sorted(set(tokens))
word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for w, i in word2idx.items()}
vocab_size = len(vocab)

print("Vocabulary Size:", vocab_size)

Vocabulary Size: 6989


In [5]:
seq_len = 5
input_sequences = []
target_words = []

for i in range(len(tokens) - seq_len):
    input_sequences.append(tokens[i:i+seq_len])
    target_words.append(tokens[i+seq_len])

X = torch.tensor([[word2idx[w] for w in seq] for seq in input_sequences])
y = torch.tensor([word2idx[w] for w in target_words])

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

print("Dataset Shape:", X.shape)

Dataset Shape: torch.Size([24729, 5])


In [6]:
class OneHotRNN(nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.input_proj = nn.Linear(vocab_size, hidden_dim)
        self.rnn = nn.RNN(hidden_dim, hidden_dim, batch_first=True)
        self.output_layer = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = torch.nn.functional.one_hot(x, num_classes=vocab_size).float()
        x = self.input_proj(x)
        out, _ = self.rnn(x)
        return self.output_layer(out)


In [7]:
class EmbeddingRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.output_layer = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        return self.output_layer(out)

In [8]:
def train_model(model, epochs=10, lr=0.005):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    losses = []

    for epoch in range(epochs):
        total_loss = 0
        model.train()

        for batch_X, batch_y in loader:
            optimizer.zero_grad()
            output = model(batch_X)
            loss = criterion(output[:, -1, :], batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(loader)
        losses.append(avg_loss)
        print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}")

    return losses


In [9]:
model_onehot = OneHotRNN(vocab_size, 128)
model_embed = EmbeddingRNN(vocab_size, 128, 128)

print("\nTraining OneHot Model\n")
loss_onehot = train_model(model_onehot, epochs=10)

print("\nTraining Embedding Model\n")
loss_embed = train_model(model_embed, epochs=20)


Training OneHot Model

Epoch 1: Loss = 7.5875
Epoch 2: Loss = 7.1129
Epoch 3: Loss = 6.6470
Epoch 4: Loss = 5.9717
Epoch 5: Loss = 5.0049
Epoch 6: Loss = 3.9178
Epoch 7: Loss = 3.0325
Epoch 8: Loss = 2.3740
Epoch 9: Loss = 1.9306
Epoch 10: Loss = 1.6362

Training Embedding Model

Epoch 1: Loss = 7.5376
Epoch 2: Loss = 6.6448
Epoch 3: Loss = 5.5398
Epoch 4: Loss = 4.3279
Epoch 5: Loss = 3.3728
Epoch 6: Loss = 2.6745
Epoch 7: Loss = 2.1364
Epoch 8: Loss = 1.7716
Epoch 9: Loss = 1.5158
Epoch 10: Loss = 1.3288
Epoch 11: Loss = 1.1967
Epoch 12: Loss = 1.1393
Epoch 13: Loss = 1.0832
Epoch 14: Loss = 1.0408
Epoch 15: Loss = 1.0119
Epoch 16: Loss = 0.9737
Epoch 17: Loss = 0.9668
Epoch 18: Loss = 0.9503
Epoch 19: Loss = 0.9423
Epoch 20: Loss = 0.9504


In [1]:
plt.plot(loss_onehot, marker='o', label="OneHot")
plt.plot(loss_embed, marker='s', label="Embedding")
plt.legend()
plt.title("Training Loss Comparison")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

NameError: name 'plt' is not defined

In [12]:
X = torch.nn.functional.one_hot(x, num_classes=vocab_size).float()


NameError: name 'x' is not defined

In [10]:
print("X exists?", 'X' in globals())


X exists? True


In [13]:
max_vocab = 5000
vocab = vocab[:max_vocab]


In [14]:
loader = DataLoader(dataset, batch_size=16, shuffle=True)


In [15]:
model_onehot = OneHotRNN(vocab_size, 64)


In [16]:
print("Vocab Size:", vocab_size)


Vocab Size: 6989


In [17]:
seq_len = 5
inputs = []
targets = []

for i in range(len(tokens) - seq_len):
    inputs.append(tokens[i:i+seq_len])
    targets.append(tokens[i+seq_len])

X = torch.tensor([[word2idx[w] for w in seq] for seq in inputs])
y = torch.tensor([word2idx[w] for w in targets])

print("Input Shape:", X.shape)
print("Target Shape:", y.shape)


Input Shape: torch.Size([24729, 5])
Target Shape: torch.Size([24729])


In [18]:
from torch.utils.data import DataLoader, TensorDataset

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [19]:
import torch.nn as nn

class TextRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc(out)
        return out


In [20]:
model = TextRNN(vocab_size, 128, 128)

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)


In [21]:
losses = []

for epoch in range(15):
    total_loss = 0
    model.train()

    for batch_X, batch_y in loader:
        optimizer.zero_grad()

        output = model(batch_X)
        loss = criterion(output[:, -1, :], batch_y)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    losses.append(avg_loss)

    print(f"Epoch {epoch+1}, Loss = {avg_loss:.4f}")


Epoch 1, Loss = 7.7386
Epoch 2, Loss = 6.7589
Epoch 3, Loss = 5.5616
Epoch 4, Loss = 4.5317
Epoch 5, Loss = 3.8472
Epoch 6, Loss = 3.3816
Epoch 7, Loss = 3.0547
Epoch 8, Loss = 2.8040
Epoch 9, Loss = 2.7050
Epoch 10, Loss = 2.5688
Epoch 11, Loss = 2.5035
Epoch 12, Loss = 2.4212
Epoch 13, Loss = 2.4076
Epoch 14, Loss = 2.3302
Epoch 15, Loss = 2.3402


In [ ]:
import matplotlib.pyplot as plt

plt.plot(losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()


In [1]:
def generate(model, start_word, length=20):
    model.eval()
    words = [start_word]

    input_seq = torch.tensor([[word2idx[start_word]]])

    for _ in range(length):
        output = model(input_seq)
        prob = torch.softmax(output[0, -1], dim=0)
        idx = torch.multinomial(prob, 1).item()

        words.append(idx2word[idx])
        input_seq = torch.tensor([[idx]])

    return " ".join(words)

print(generate(model, "love"))


NameError: name 'model' is not defined